# The Battle of the Neighborhoods - Week 2

### Applied Data Science Capstone

## Table of contents
* [Businesss Problem Introduction](#introduction)
* [Data Preparation & Data Cleaning](#data)
* [Methodology & Data Analysis](#methodology)
* [Results and Discussion](#results)
* [Conclusion](#conclusion)

## Business Problem Introduction

In this project we will try to find an optimal locations for book stores. Our company are looking for optimal locations to open book stores in mutiple cities in the US based on the demographic information and geographic information.

First, we will try to find one optimal city as a starting location and baseing on the school or population information to decide which zip code to open the store.

In [1]:
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


Connect with the Foursquare API

In [2]:
client_id = 'HUN1ZHQZV0VCKJ2ZTU3V53VWZ521E1R05A4J5V4QVIQKW2U2' # your Foursquare ID
client_secret = 'SUBCK3OZ4O4E3MBQLLLRXF2CPFI051V4NTSIBY0I24FYHCB0' # your Foursquare Secret
version = '20180605' # Foursquare API version

bookstore_id = '4bf58dd8d48988d114951735'
schools_ids = '4bf58dd8d48988d13b941735'

In [3]:
limit = 500 # Maximum is 100
# We have first selected 5 cities
cities = ["Miami, FL", "New York, NY", 'Chicago, IL', 'San Francisco, CA', 'Seattle, WA']
results = {}
for city in cities:
    url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&near={}&limit={}&categoryId={}'.format(
        client_id, 
        client_secret, 
        version, 
        city,
        limit,
        bookstore_id) # Book stores id
    results[city] = requests.get(url).json()

In [4]:
df_venues={}
for city in cities:
    venues = pd.json_normalize(results[city]['response']['groups'][0]['items'])
    df_venues[city] = venues[['venue.name', 'venue.location.address', 'venue.location.lat', 'venue.location.lng']]
    df_venues[city].columns = ['Name', 'Address', 'Lat', 'Lng']

In [5]:
maps = {}
for city in cities:
    city_lat = np.mean([results[city]['response']['geocode']['geometry']['bounds']['ne']['lat'],
                        results[city]['response']['geocode']['geometry']['bounds']['sw']['lat']])
    city_lng = np.mean([results[city]['response']['geocode']['geometry']['bounds']['ne']['lng'],
                        results[city]['response']['geocode']['geometry']['bounds']['sw']['lng']])
    maps[city] = folium.Map(location=[city_lat, city_lng], zoom_start=11)

    # add markers to map
    for lat, lng, label in zip(df_venues[city]['Lat'], df_venues[city]['Lng'], df_venues[city]['Name']):
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            popup=label,
            color='blue',
            fill=True,
            fill_color='#3186cc',
            fill_opacity=0.7,
            parse_html=False).add_to(maps[city])  
    print(f"Total number of book stores in {city} = ", results[city]['response']['totalResults'])
    print("Showing Top 100")

Total number of book stores in Miami, FL =  67
Showing Top 100
Total number of book stores in New York, NY =  216
Showing Top 100
Total number of book stores in Chicago, IL =  148
Showing Top 100
Total number of book stores in San Francisco, CA =  152
Showing Top 100
Total number of book stores in Seattle, WA =  129
Showing Top 100


As we can see, New York and Chicago have more book stores.

Let's take a close look of them and see the distance from the central area for each city.

In [6]:
maps = {}
for city in cities:
    city_lat = np.mean([results[city]['response']['geocode']['geometry']['bounds']['ne']['lat'],
                        results[city]['response']['geocode']['geometry']['bounds']['sw']['lat']])
    city_lng = np.mean([results[city]['response']['geocode']['geometry']['bounds']['ne']['lng'],
                        results[city]['response']['geocode']['geometry']['bounds']['sw']['lng']])
    maps[city] = folium.Map(location=[city_lat, city_lng], zoom_start=11)
    venues_mean_coor = [df_venues[city]['Lat'].mean(), df_venues[city]['Lng'].mean()] 
    # add markers to map
    for lat, lng, label in zip(df_venues[city]['Lat'], df_venues[city]['Lng'], df_venues[city]['Name']):
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            popup=label,
            color='blue',
            fill=True,
            fill_color='#3186cc',
            fill_opacity=0.7,
            parse_html=False).add_to(maps[city])
        folium.PolyLine([venues_mean_coor, [lat, lng]], color="green", weight=1.5, opacity=0.5).add_to(maps[city])
    
    label = folium.Popup("Mean Co-ordinate", parse_html=True)
    folium.CircleMarker(
        venues_mean_coor,
        radius=10,
        popup=label,
        color='green',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(maps[city])

    print(city)
    print("Mean Distance from Mean coordinates")
    print(np.mean(np.apply_along_axis(lambda x: np.linalg.norm(x - venues_mean_coor),1,df_venues[city][['Lat','Lng']].values)))

Miami, FL
Mean Distance from Mean coordinates
0.13907263612601747
New York, NY
Mean Distance from Mean coordinates
0.028738798464095376
Chicago, IL
Mean Distance from Mean coordinates
0.0814520755200635
San Francisco, CA
Mean Distance from Mean coordinates
0.030325595084376263
Seattle, WA
Mean Distance from Mean coordinates
0.04411514115570099


People live in Miami need to travel the longest distance to a book store, then Chicago. This makes sense to me, because in Miami and Chicago, everyone drives a lot. People are willing to drive to buy books if they need to.

First, let's take a look of the book stores in New York

In [7]:
maps[cities[1]]

Basing on the density we can tell most of the book stores are located in Manhattan and near the universities, however the rent in Manhattan are so high. And for textbooks, the price are fixed, the return rate may not very high for this area.

Let's take a close look of Chicago instead.

In [8]:
maps[cities[2]]

Book stores in Chicago are more separated, which there are higher chance for us to find a good location with lower rent and high density of population. 

Let's look into Chicago. 

First, let's check how many schools are in Chicago and how far are they from the central.

In [9]:
limit = 500 
selected_city = ['Chicago, IL']
results = {}
for city in selected_city:
    url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&near={}&limit={}&categoryId={}'.format(
        client_id, 
        client_secret, 
        version, 
        city,
        limit,
        schools_ids)
    results[city] = requests.get(url).json()

In [10]:
df_venues_s={}
for city in selected_city:
    venues = pd.json_normalize(results[city]['response']['groups'][0]['items'])
    df_venues_s[city] = venues[['venue.name', 'venue.location.address', 'venue.location.lat', 'venue.location.lng']]
    df_venues_s[city].columns = ['Name', 'Address', 'Lat', 'Lng']

In [11]:
maps[selected_city[0]]

In [12]:
maps = {}
for city in selected_city:
    city_lat = np.mean([results[city]['response']['geocode']['geometry']['bounds']['ne']['lat'],
                        results[city]['response']['geocode']['geometry']['bounds']['sw']['lat']])
    city_lng = np.mean([results[city]['response']['geocode']['geometry']['bounds']['ne']['lng'],
                        results[city]['response']['geocode']['geometry']['bounds']['sw']['lng']])
    maps[city] = folium.Map(location=[city_lat, city_lng], zoom_start=11)

    # add markers to map
    for lat, lng, label in zip(df_venues_s[city]['Lat'], df_venues_s[city]['Lng'], df_venues_s[city]['Name']):
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            popup=label,
            color='blue',
            fill=True,
            fill_color='#3186cc',
            fill_opacity=0.7,
            parse_html=False).add_to(maps[city])  
    print(f"Total number of schools in {city} = ", results[city]['response']['totalResults'])

Total number of schools in Chicago, IL =  270


In [13]:
df_venues_s[city].columns = [col.split(".")[-1] for col in df_venues_s[city].columns]
df_venues_s[city].head()

,Name,Address,Lat,Lng
0,It Takes A Village,800 S Wells St Ste 180,41.872199,-87.633867
1,Paul Mitchell the School,1225 S Halsted St,41.866190,-87.646712
2,St. Benedict Preparatory Schools,3900 N Leavitt St,41.952862,-87.684460
3,Chicago Westside christian school,NaN,41.865036,-87.725229
4,Horizon Math & Science Academy Southwest,5401 S Western Blvd,41.795171,-87.681110


In [14]:
df_venues[city].columns = [col.split(".")[-1] for col in df_venues[city].columns]
df_venues[city].head()

,Name,Address,Lat,Lng
0,Unabridged Books,3251 N Broadway St,41.941588,-87.644386
1,Myopic Books,1564 N Milwaukee Ave,41.909977,-87.676851
2,57th Street Books,1101 E 57th St,41.791469,-87.594591
3,The Book Cellar,4736 N Lincoln Ave,41.967656,-87.687868
4,Open Books,651 W Lake St,41.885719,-87.644872


In [15]:
df_venues[city].to_csv('C:/Users/lisal/Desktop/bookstores.csv')
df_venues_s[city].to_csv('C:/Users/lisal/Desktop/schools.csv')

Now, let's calculate the distance for each school 

As we can see above, the locations of the book stores kind of matching the locations of schools.

Let's calculate the distance for each schools to the book stores and find the book stores within 2 miles radius

Basing the school zip code, we can get the population from Census and find the schools with top 5 population and with less than 5 book stores within 2 miles radius 

Since I already have ready code in R for the calculation.

I will do the calculation part in R instead.

Use this Link for R code: 

Please see the results below. 

In [22]:
import csv
bookstorelist = csv.DictReader(open('C:/Users/lisal/Desktop/bookstores.csv'))
bookstorelist = pd.DataFrame(bookstorelist)
bookstorelist.head()

,,Num,Name,Address,Lng,Lat,meter,kilometer,miles,school_name
0,1,4,Open Books,651 W Lake St,41.8857188858997,-87.6448720266537,1757.69706036205,1.75769706036205,1.09218198009423,It Takes A Village
1,2,12,Sandmeyer's Bookstore,714 S Dearborn St,41.8729960533556,-87.6292867926672,390.400842816317,0.390400842816317,0.242583762101618,It Takes A Village
2,3,13,Selected Works Used Books & Sheet Music,410 South Michigan Avenue,41.8765292651336,-87.6243478218776,925.084526910335,0.925084526910335,0.574820697570802,It Takes A Village
3,4,22,Pilsen Community Book Shop,1102 W 18th St,41.858009057714,-87.6536723536155,2277.70053942623,2.27770053942623,1.41529706188382,It Takes A Village
4,5,23,After-Words New & Used Books,23 E Illinois St,41.8909470290503,-87.627137973962,2155.97182324206,2.15597182324206,1.33965836777974,It Takes A Village


Be aware due to the data limit foursquare, we can't do a full analysis on a real case.

So instead, we can take the school `It takes a village` as an example to see the demographic information in this Zip code.

In [23]:
import csv
demoinfo = csv.DictReader(open('C:/Users/lisal/Desktop/demoinfo.csv'))
demoinfo = pd.DataFrame(demoinfo)
demoinfo.head()

,,zip,zip_full,median_income,median_age,total_population,male_population,female_population,white_alone,black_alone,american_indian_alaska_native_alone,asian_alone,pacific_islander_alone,other_race,hispanic_latino
0,1,60651,ZCTA5 60651,38083,33,63218,29134,34084,10554,32913,168,306,0,17537,26860


Conclusion: 

As we can see, in 2 miles radius for each school, there are 24 bookstores for school named `It takes a village`

Based on the competitors, median income and median age, etc information. 

Not really sure whether this could be a good location to open a bookstores. Need to get full list to do accurate calculation.